In [124]:
import pandas as pd

# Read the featuremaps clusters data
featuremaps_df = pd.read_pickle('../logs/feature_combinations_clusters').reset_index()
# Compute the total time
featuremaps_df['time'] = featuremaps_df['map_time'] + featuremaps_df['features_extraction']
featuremaps_df = featuremaps_df.drop(columns=['map_time', 'features_extraction'])

# Get the original clusters
featuremaps_original_clusters_df = pd.DataFrame(data = featuremaps_df[['approach', 'clusters_list', 'time']].values, columns=['approach', 'clusters', 'time'])
featuremaps_original_clusters_df['approach'] = featuremaps_original_clusters_df['approach'].apply(lambda approach: approach + '_original')
# Get the merged clusters
featuremaps_merged_clusters_df = pd.DataFrame(data = featuremaps_df[['approach', 'merged_clusters_list', 'time']].values, columns=['approach', 'clusters', 'time'])
featuremaps_merged_clusters_df['approach'] = featuremaps_merged_clusters_df['approach'].apply(lambda approach: approach + '_merged')
# Add the clusters
featuremaps_all_clusters_df = pd.concat([featuremaps_original_clusters_df, featuremaps_merged_clusters_df]).reset_index(drop=True)
# Get the number of clusters
featuremaps_all_clusters_df['num_clusters'] = featuremaps_all_clusters_df['clusters'].apply(len)

featuremaps_all_clusters_df

,approach,clusters,time,num_clusters
0,moves+orientation(10x10)_original,"[[28, 119, 263, 277, 339, 453], [20, 38, 72, 8...",26.846361,44
1,moves+bitmaps(10x10)_original,"[[670], [48, 163, 252, 254, 274, 279, 401, 426...",26.817801,47
2,orientation+bitmaps(10x10)_original,"[[28, 339, 453], [119, 263, 277], [248], [252,...",26.84524,55
3,moves+orientation(20x20)_original,"[[339], [28, 263, 277, 453], [119, 125, 210, 2...",27.827787,76
4,moves+bitmaps(20x20)_original,"[[670], [48, 279, 520], [252, 254, 274, 426, 4...",27.795271,76
5,orientation+bitmaps(20x20)_original,"[[339], [28, 453], [263], [277], [125], [119, ...",27.971384,169
6,moves+orientation(10x10)_merged,"[[28, 119, 263, 277, 339, 453, 20, 38, 72, 83,...",26.846361,3
7,moves+bitmaps(10x10)_merged,"[[670, 48, 163, 252, 254, 274, 279, 401, 426, ...",26.817801,4
8,orientation+bitmaps(10x10)_merged,"[[28, 339, 453, 119, 263, 277, 248, 252, 520, ...",26.84524,1
9,moves+orientation(20x20)_merged,"[[339, 28, 263, 277, 453, 119, 125, 210, 289, ...",27.827787,26


In [125]:
import pandas as pd
import ast

# Read the heatmaps data
heatmaps_df = pd.read_csv('../logs/heatmaps_data.csv')
heatmaps_df['clusters'] = heatmaps_df['clusters'].apply(ast.literal_eval)

# Extract the clusters df
heatmaps_clusters_df = heatmaps_df[['clustering_mode', 'explainer', 'clustering_technique', 'clusters', 'num_clusters', 'time_clustering', 'time_contributions']]
heatmaps_clusters_df['clustering_mode'] = heatmaps_clusters_df.apply(lambda row: f'{row["clustering_technique"]}({row["clustering_mode"]})', axis=1)
heatmaps_clusters_df = heatmaps_clusters_df.drop(columns=['clustering_technique'])
heatmaps_clusters_df = heatmaps_clusters_df.rename(columns={'explainer': 'approach'})
heatmaps_clusters_df['time'] = heatmaps_clusters_df['time_clustering'] + heatmaps_clusters_df['time_contributions']
heatmaps_clusters_df = heatmaps_clusters_df.drop(columns=['time_clustering', 'time_contributions'])

heatmaps_clusters_df.head()

/var/folders/b2/ybzqqdk538s60ppvpsh_9q2m0000gn/T/ipykernel_52695/3220510469.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heatmaps_clusters_df['clustering_mode'] = heatmaps_clusters_df.apply(lambda row: f'{row["clustering_technique"]}({row["clustering_mode"]})', axis=1)


,clustering_mode,approach,clusters,num_clusters,time
0,AffinityPropagation(LocalLatentMode),GradCAM,"[[0, 385, 258, 3, 389, 146, 25, 282, 409, 161,...",25,3.87692
1,AffinityPropagation(LocalLatentMode),GradCAM,"[[0, 385, 258, 3, 389, 146, 25, 282, 409, 161,...",25,3.21302
2,AffinityPropagation(LocalLatentMode),GradCAM,"[[0, 385, 130, 519, 523, 19, 24, 156, 286, 164...",27,3.61985
3,AffinityPropagation(LocalLatentMode),GradCAM,"[[0, 385, 258, 389, 146, 282, 161, 420, 179, 3...",26,3.88618
4,AffinityPropagation(LocalLatentMode),GradCAM,"[[0, 385, 258, 389, 136, 146, 409, 282, 31, 16...",28,3.34055


In [126]:
from utils.dataset import get_train_test_data, get_data_masks
import pandas as pd
import numpy as np

# Merge all the clusters together
complete_df = pd.concat([featuremaps_all_clusters_df, heatmaps_clusters_df]).reset_index(drop=True)

# Extract data about the clusters sizes
complete_df['clusters_sizes'] = complete_df['clusters'].apply(lambda clusters: [len(cluster) for cluster in clusters])

# Get the indexes of the misclassified elements
(train_data, train_labels), (test_data, test_labels) = get_train_test_data(rgb=True)
predictions = np.loadtxt('../in/predictions.csv')
mask_miss, mask_label = get_data_masks(test_labels, predictions, label=5)
mask_miss_label = mask_miss[mask_label]
misclassified_idxs = np.argwhere(mask_miss_label == True)
# Find the fraction of misclassified data in each cluster
complete_df['frac_misses'] = complete_df['clusters'].apply(
    lambda clusters: [
        len([entry for entry in cluster if entry in misclassified_idxs]) / len(cluster)
        for cluster in clusters
    ]
)
# Find the fraction of clusters containing both correct and incorrect classifications
complete_df['frac_mixed'] = complete_df['frac_misses'].apply(lambda misses: len([entry for entry in misses if 0 < entry < 1]) / len(misses))

complete_df

,approach,clusters,time,num_clusters,clustering_mode,clusters_sizes,frac_misses,frac_mixed
0,moves+orientation(10x10)_original,"[[28, 119, 263, 277, 339, 453], [20, 38, 72, 8...",26.846361,44,NaN,"[6, 41, 76, 115, 118, 220, 176, 42, 7, 1, 1, 2...","[0.16666666666666666, 0.4146341463414634, 0.31...",0.431818
1,moves+bitmaps(10x10)_original,"[[670], [48, 163, 252, 254, 274, 279, 401, 426...",26.817801,47,NaN,"[1, 15, 90, 222, 208, 148, 90, 21, 6, 1, 1, 1,...","[1.0, 0.4666666666666667, 0.3111111111111111, ...",0.319149
2,orientation+bitmaps(10x10)_original,"[[28, 339, 453], [119, 263, 277], [248], [252,...",26.84524,55,NaN,"[3, 3, 1, 2, 3, 16, 16, 3, 2, 5, 26, 15, 24, 9...","[0.3333333333333333, 0.0, 0.0, 1.0, 0.66666666...",0.709091
3,moves+orientation(20x20)_original,"[[339], [28, 263, 277, 453], [119, 125, 210, 2...",27.827787,76,NaN,"[1, 4, 8, 23, 31, 43, 48, 52, 58, 56, 72, 108,...","[1.0, 0.0, 0.625, 0.391304347826087, 0.4193548...",0.250000
4,moves+bitmaps(20x20)_original,"[[670], [48, 279, 520], [252, 254, 274, 426, 4...",27.795271,76,NaN,"[1, 3, 10, 19, 63, 77, 98, 107, 116, 82, 66, 6...","[1.0, 1.0, 0.3, 0.5789473684210527, 0.25396825...",0.263158
5,orientation+bitmaps(20x20)_original,"[[339], [28, 453], [263], [277], [125], [119, ...",27.971384,169,NaN,"[1, 2, 1, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 4, 4, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.66666666...",0.538462
6,moves+orientation(10x10)_merged,"[[28, 119, 263, 277, 339, 453, 20, 38, 72, 83,...",26.846361,3,NaN,"[890, 1, 1]","[0.18764044943820224, 0.0, 0.0]",0.333333
7,moves+bitmaps(10x10)_merged,"[[670, 48, 163, 252, 254, 274, 279, 401, 426, ...",26.817801,4,NaN,"[889, 1, 1, 1]","[0.18672665916760406, 1.0, 0.0, 0.0]",0.250000
8,orientation+bitmaps(10x10)_merged,"[[28, 339, 453, 119, 263, 277, 248, 252, 520, ...",26.84524,1,NaN,[892],[0.18721973094170405],1.000000
9,moves+orientation(20x20)_merged,"[[339, 28, 263, 277, 453, 119, 125, 210, 289, ...",27.827787,26,NaN,"[801, 1, 1, 4, 1, 1, 1, 1, 3, 1, 5, 1, 3, 1, 5...","[0.18976279650436953, 1.0, 0.0, 0.5, 0.0, 1.0,...",0.192308
